## На основе алгоритма Флойда. Пересчет таблицы "кратчайших маршрутов" только если были изменения в вершинах, и связях.

In [ ]:
from math import inf


class Descriptor:

    def __set_name__(self, owner, name) -> None:
        self.name = "_" + name

    def __get__(self, instance, owner) -> (int, property):
        if instance is None:
            return property()
        return getattr(instance, self.name)

    def __set__(self, instance, value):
        setattr(instance, self.name, value)


class Vertex:

    links = Descriptor()

    def __init__(self) -> None:
        self.links: list = []


class Link:
    v1 = Descriptor()
    v2 = Descriptor()
    dist = Descriptor()

    def __init__(self, v1: Vertex, v2: Vertex, distance: int=1) -> None:
        self.v1 = v1
        self.v2 = v2
        self.dist = distance
        v1.links.append(self)
        v2.links.append(self)

    def __eq__(self, other):
        return {self.v1, self.v2} == {other.v1, other.v2}

    def __str__(self):
        return self.__class__.__name__ + " : " + str(self.v1) + " - " + str(self.v2)

    def __repr__(self):
        return self.__str__()


class LinkedGraph:
    links = Descriptor()
    vertex = Descriptor()

    def __init__(self):
        self.links: list = []
        self.vertex: list = []

    def add_vertex(self, vertex):
        if vertex not in self.vertex:
            self.vertex.append(vertex)

    def add_link(self, link):
        if link not in self.links:
            self.add_vertex(link.v1)
            self.add_vertex(link.v2)
            self.links.append(link)

    def find_path(self, stop_v, start_v):
        v_dct = {v: k for k, v in enumerate(self.vertex)}
        dct_v = {k: v for k, v in enumerate(self.vertex)}
        v_mat = [[inf for _ in range(len(v_dct))] for __ in range(len(v_dct))] # матрица смежности
        for i in self.links:
            v_mat[v_dct.get(i.v1)][v_dct.get(i.v2)] = i.dist
            v_mat[v_dct.get(i.v2)][v_dct.get(i.v1)] = i.dist
            v_mat[v_dct.get(i.v1)][v_dct.get(i.v1)] = 0
            v_mat[v_dct.get(i.v2)][v_dct.get(i.v2)] = 0

        N = len(v_dct)  # число вершин в графе
        P = [[z for z in range(N)] for k in range(N)]  # начальный список предыдущих вершин для поиска кратчайших маршрутов

        for k in range(N):
            for i in range(N):
                for j in range(N):
                    d = v_mat[i][k] + v_mat[k][j]
                    if v_mat[i][j] > d:
                        v_mat[i][j] = d
                        P[i][j] = k  # номер промежуточной вершины при движении от i к j

        path = [start_v]
        start_v_tmp = start_v
        while start_v_tmp != stop_v:

            start_v_tmp = dct_v.get(P[v_dct.get(start_v_tmp)][v_dct.get(stop_v)])
            path.append(start_v_tmp)

        path = list(reversed(path))
        path_links = []
        for k, i in enumerate(path):
            if k + 1 < len(path):
                path_links.append(LinkMetro(i, path[k+1], v_mat[v_dct.get(i)][v_dct.get(path[k+1])]))
        return path, path_links


class Station(Vertex):

    def __init__(self, name: str):
        super(Station, self).__init__()
        self.name = name

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name


class LinkMetro(Link): pass


# map_metro = LinkedGraph()
# v1 = Station("Сретенский бульвар")
# v2 = Station("Тургеневская")
# v3 = Station("Чистые пруды")
# v4 = Station("Лубянка")
# v5 = Station("Кузнецкий мост")
# v6 = Station("Китай-город 1")
# v7 = Station("Китай-город 2")
#
# map_metro.add_link(LinkMetro(v1, v2, 1))
# map_metro.add_link(LinkMetro(v2, v3, 1))
# map_metro.add_link(LinkMetro(v1, v3, 1))
#
# map_metro.add_link(LinkMetro(v4, v5, 1))
# map_metro.add_link(LinkMetro(v6, v7, 1))
#
# map_metro.add_link(LinkMetro(v2, v7, 5))
# map_metro.add_link(LinkMetro(v3, v4, 3))
# map_metro.add_link(LinkMetro(v5, v6, 3))
#
# # print(map_metro.links)
# # print(map_metro.vertex)
# # map_metro.find_path(v1, v2)
#
#
# path = map_metro.find_path(v1, v6)  # от сретенского бульвара до китай-город 1
# print(path[0])    # [Сретенский бульвар, Тургеневская, Китай-город 2, Китай-город 1]
# # print(path[1])
# print(sum([x.dist for x in path[1]]))  # 7

In [ ]:
import math
import copy
# на основе алгоритма Флойда (Floyd's algorithm) по https://www.youtube.com/watch?v=ipWZ-d1l00s&ab_channel=selfedu
# Использован код С. Балакирева https://github.com/selfedu-rus/python-algorithms/blob/master/algorithm-floyd.py

class Vertex:
	'''для представления вершин графа (на карте это могут быть: здания, остановки, достопримечательности и т.п.)'''
	_ID = -1
	def __new__(cls, *args, **kwargs):
		cls._ID += 1
		return super().__new__(cls)
	def __init__(self):
		self._id = self._ID
		self._links = []

	@property
	def links(self):
		return self._links

class Link:
	'''для описания связи между двумя произвольными вершинами графа (на карте: маршруты, время в пути и т.п.)'''
	def __init__(self, v1, v2, dist=1):
		self._v1 = v1
		self._v2 = v2
		self.dist = dist

	@property
	def v1(self):
		return self._v1
	@property
	def v2(self):
		return self._v2
	@property
	def dist(self):
		return self._dist
	@dist.setter
	def dist(self, dist):
		self._dist = dist
	def __setattr__(self, key, value):
		if key in ('_v1', '_v2') and not isinstance(value, Vertex):
			raise ValueError('недопустимое значение аргумента')
		object.__setattr__(self, key, value)

class LinkedGraph:
	'''для представления связного графа в целом (карта целиком)'''
	_ID = 0
	def __init__(self):
		self._links = []
		self._vertex = []
		self._m_graph = []
		self._m_result = []
		#self.__ID = 0
	def add_vertex(self, v):
		if v not in self._vertex:
			self._vertex.append(v)
			if v._id > self._ID:
				self._ID = v._id
	def add_link(self, link):
		if sorted((link.v1._id, link.v2._id)) not in list(sorted((l.v1._id, l.v2._id)) for l in self._links):
			self._links.append(link)
			self.add_vertex(link.v1) 
			self.add_vertex(link.v2)
			link.v1.links.append(link)
			link.v2.links.append(link)

	def find_path(self, start_v, stop_v):  #  def get_path(P, u, v)
		V = self.matrix_graph()
		if V != self._m_graph:
			self._m_graph = copy.deepcopy(V)
			self._m_result = self.matrix_result(V)
		
		v, u = start_v._id, stop_v._id
		path = [u]
		while u != v:
			u = self._m_result[u][v]
			path.append(u)
		return self.result(path, start_v)
	
	def matrix_result(self, V):
		'''построение таблицы кратчайших маршрутовпо Алгоритму Флойда'''
		N = len(V)   # число вершин в графе
		P = [[v for v in range(N)] for u in range(N)]    # начальный список предыдущих вершин для поиска кратчайших маршрутов
		for k in range(N):
			for i in range(N):
				for j in range(N):
					d = V[i][k] + V[k][j]
					if V[i][j] > d:
						V[i][j] = d
						P[i][j] = k	
		return P
	
	def matrix_graph(self):
		'''функция построения карты смежности'''
		m_graph = list(list(0 if i == j else math.inf for j in range(self._ID+1)) for i in range (self._ID+1)) 

		for v in self._vertex:			
			for l in v.links:
				if v._id == l.v1._id: m_graph[v._id][l.v2._id] = l.dist					
				if v._id == l.v2._id: m_graph[v._id][l.v1._id] = l.dist							
		return m_graph	

	def result(self, p, start_v):
		'''функция представления результата расчета согласно техзаданию. 
		Кортеж ([вершины кратчайшего пути], [связи между вершинами]) '''
		res1 = []
		res2 = []
		for i in range(len(p) - 1):
			for obj in self._vertex:
				if obj._id == p[i]:
					res1 = [obj] + res1
			for ln in self._links:
				if sorted((ln.v1._id, ln.v2._id)) == sorted((p[i], p[i+1])):
					res2 = [ln] + res2
		return [start_v] + res1, res2
	
class Station(Vertex):
	def __init__(self, name):
		super().__init__()
		self.name = name
	def __str__(self):
		return self.name
	def __repr__(self):
		return self.__str__()

class LinkMetro(Link):
	def __init__(self, v1, v2, dist):
		super().__init__(v1, v2, dist)